In [1]:
import os
import pandas as pd
from paths import *
from misc_tools import PathUtils as PU
from misc_tools import AudioCut

In [2]:
def assign_syllable_id(group):
    # Initialize syllable_id
    syllable_id = 0
    syllable_ids = []
    
    # Iterate over the rows in the group
    for index, row in group.iterrows():
        if row['syllable_in_id'] == 1:
            # Increment syllable_id when syllable_in_id is 1
            syllable_id += 1
        syllable_ids.append(syllable_id)
    
    # Add the syllable_ids to the group
    group['syllable_id'] = syllable_ids
    return group

In [3]:
for guide_name in ["guide_validation"]: # "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    guide_ = os.path.join(src_eng_, f"{guide_name}_original.csv")
    guide_df = pd.read_csv(guide_)
    guide_df = guide_df.groupby('file').apply(assign_syllable_id).reset_index(drop=True)
    guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}.csv"), index=False)

# Append Syllable UID and Path

In [20]:
for x_name in ["guide_validation.csv"]: # "guide_train", "guide_validation.csv", "guide_test.csv"
    x_ = os.path.join(src_eng_, x_name)

    guide_x = pd.read_csv(x_)

    guide_x["syllable_path"] = guide_x.apply(AudioCut.syllablerecord2filepath, axis=1)

    guide_x["suid"] = guide_x.apply(AudioCut.syllablerecord2suid, axis=1)

    guide_x.to_csv(x_, index=False)

In [2]:
guide_ = os.path.join(src_eng_, "guide_test_complete.csv")
guide_df = pd.read_csv(guide_)

# Add stress mark

In [21]:
import re

In [ ]:
for guide_name in ["guide_validation"]: # "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    guide_ = os.path.join(src_eng_, f"{guide_name}.csv")
    guide_df = pd.read_csv(guide_)
    guide_df = guide_df.groupby('file').apply(assign_syllable_id).reset_index(drop=True)
    guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}.csv"), index=False)

# Append nSample

In [21]:
import torchaudio

In [22]:
def get_nSample(row): 
    path = row["syllable_path"]
    try: 
        waveform, sample_rate = torchaudio.load(os.path.join(train_cut_syllable_, path))
        return waveform.size(-1)
    except Exception: 
        return 0

In [23]:
for guide_name in ["guide_validation", "guide_test", "guide_train"]: # "guide_train" "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    print(guide_name)
    guide_ = os.path.join(src_eng_, f"{guide_name}.csv")
    guide_df = pd.read_csv(guide_)
    small_df = guide_df[["syllable_path", "suid"]].groupby('suid').first().reset_index()
    small_df["syllable_nSample"] = small_df.apply(get_nSample, axis=1)
    small_df = small_df.drop(columns=["syllable_path"])
    large_guide_df = pd.merge(guide_df, small_df, how="left", on=["suid"])
    large_guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}_mod.csv"), index=False)

guide_validation
guide_test
guide_train


In [27]:
guide_ = os.path.join(src_eng_, "guide_train_mod.csv")
guide_df = pd.read_csv(guide_)

In [28]:
test_df = guide_df.groupby('suid').first().reset_index()

In [29]:
len(test_df[test_df["syllable_nSample"] == 0]) / len(test_df)

2.7818707338853184e-06

In [14]:
recpath = guide_df.iloc[0]["syllable_path"]

In [15]:
get_nSample(os.path.join(train_cut_syllable_, recpath))

10720

# Add stress mark

In [32]:
import re

In [33]:
def getSyllableStressType(row): 
    syllable = row["syllable"]
    stresses = re.findall(r'\d+', syllable)
    if len(stresses) != 1: 
        return "SNA"
    else: 
        return stresses[0]

In [34]:
for guide_name in ["guide_train", "guide_validation", "guide_test"]: # "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    guide_ = os.path.join(src_eng_, f"{guide_name}_mod.csv")
    guide_df = pd.read_csv(guide_)
    small_df = guide_df[["syllable", "suid"]].groupby('suid').first().reset_index()
    small_df["syllable_stress_type"] = small_df.apply(getSyllableStressType, axis=1)
    small_df = small_df.drop(columns=["syllable"])
    large_guide_df = pd.merge(guide_df, small_df, how="left", on=["suid"])
    large_guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}_mod.csv"), index=False)

In [36]:
large_guide_df.dtypes

index                     int64
segment                  object
file                     object
id                        int64
startTime               float64
endTime                 float64
nSample                   int64
word_id                 float64
word                     object
in_id                   float64
segment_nostress         object
stress_type              object
phone_path               object
word_path                object
speaker                   int64
word_startTime          float64
word_endTime            float64
word_nSample            float64
wuid                     object
syllable                 object
syllable_in_id          float64
syllable_startTime      float64
syllable_endTime        float64
syllable_id               int64
syllable_path            object
suid                     object
syllable_nSample          int64
syllable_stress_type     object
dtype: object

In [65]:
guide_ = os.path.join(src_eng_, f"guide_train_mod.csv")
guide_df = pd.read_csv(guide_)

In [66]:
guide_df.iloc[0]

index                                                             1
segment                                                          CH
file                                               1034-121119-0000
id                                                                1
startTime                                                      0.19
endTime                                                        0.37
nSample                                                        2880
word_id                                                         0.0
word                                                        chapter
in_id                                                           1.0
segment_nostress                                                 CH
stress_type                                                     SNA
phone_path              1034/121119/0000/1034-121119-0000-0001.flac
word_path               1034/121119/0000/1034-121119-0000-0000.flac
speaker                                         

In [5]:
import torchaudio
from model_dataset import SyllableDataset, TokenMap
from model_configs import ModelDimConfigs, TrainingConfigs
from misc_tools import get_timestamp, ARPABET
from model_dataset import DS_Tools, Padder, TokenMap, NormalizerKeepShape
from model_filter import XpassFilter
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split

In [38]:
select = ["0", "1", "2"]
mymap = TokenMap(select)

In [7]:
mytrans = nn.Sequential(
    Padder(sample_rate=TrainingConfigs.REC_SAMPLE_RATE, pad_len_ms=250, noise_level=1e-4), 
    XpassFilter(cut_off_upper=500),
    torchaudio.transforms.MelSpectrogram(TrainingConfigs.REC_SAMPLE_RATE, 
                                        n_mels=TrainingConfigs.N_MELS, 
                                        n_fft=TrainingConfigs.N_FFT, 
                                        power=2), 
    torchaudio.transforms.AmplitudeToDB(stype="power", top_db=80), 
    NormalizerKeepShape(NormalizerKeepShape.norm_mvn)
)

In [34]:
guide_ = os.path.join(src_eng_, "guide_train_mod.csv")
guide_file = pd.read_csv(guide_)

In [37]:
guide_file["syllable_stress_type"] = guide_file["syllable_stress_type"].astype(str)

In [39]:
guide_file = guide_file[guide_file["syllable_stress_type"].isin(select)]

In [40]:
guide_file

,index,segment,file,id,startTime,endTime,nSample,word_id,word,in_id,...,wuid,syllable,syllable_in_id,syllable_startTime,syllable_endTime,syllable_id,syllable_path,suid,syllable_nSample,syllable_stress_type
0,1,CH,1034-121119-0000,1,0.19,0.37,2880,0.0,chapter,1.0,...,1034-121119-0000-0000,CH AE1 P,1.0,0.19,0.57,1,1034/121119/0000/1034-121119-0000-0001.flac,1034-121119-0000-0001,6080,1
1,2,AE1,1034-121119-0000,2,0.37,0.45,1280,0.0,chapter,2.0,...,1034-121119-0000-0000,CH AE1 P,2.0,0.19,0.57,1,1034/121119/0000/1034-121119-0000-0001.flac,1034-121119-0000-0001,6080,1
2,3,P,1034-121119-0000,3,0.45,0.57,1920,0.0,chapter,3.0,...,1034-121119-0000-0000,CH AE1 P,3.0,0.19,0.57,1,1034/121119/0000/1034-121119-0000-0001.flac,1034-121119-0000-0001,6080,1
3,4,T,1034-121119-0000,4,0.57,0.63,960,0.0,chapter,4.0,...,1034-121119-0000-0000,T ER0,1.0,0.57,0.74,2,1034/121119/0000/1034-121119-0000-0002.flac,1034-121119-0000-0002,2720,0
4,5,ER0,1034-121119-0000,5,0.63,0.74,1760,0.0,chapter,5.0,...,1034-121119-0000-0000,T ER0,2.0,0.57,0.74,2,1034/121119/0000/1034-121119-0000-0002.flac,1034-121119-0000-0002,2720,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768872,2923415,AH1,911-130578-0020,76,7.70,7.74,640,22.0,some,2.0,...,911-130578-0020-0022,S AH1 M,2.0,7.62,7.81,27,911/130578/0020/911-130578-0020-0027.flac,911-130578-0020-0027,3040,1
2768873,2923416,M,911-130578-0020,77,7.74,7.81,1120,22.0,some,3.0,...,911-130578-0020-0022,S AH1 M,3.0,7.62,7.81,27,911/130578/0020/911-130578-0020-0027.flac,911-130578-0020-0027,3040,1
2768874,2923417,F,911-130578-0020,78,7.81,7.97,2560,23.0,food,1.0,...,911-130578-0020-0023,F UW1 D,1.0,7.81,8.29,28,911/130578/0020/911-130578-0020-0028.flac,911-130578-0020-0028,7680,1
2768875,2923418,UW1,911-130578-0020,79,7.97,8.18,3360,23.0,food,2.0,...,911-130578-0020-0023,F UW1 D,2.0,7.81,8.29,28,911/130578/0020/911-130578-0020-0028.flac,911-130578-0020-0028,7680,1


In [ ]:
guide_file = guide_file[guide_file["syllable_stress_type"].isin(select)]
guide_file = guide_file[guide_file['syllable_nSample'] > 400]
# guide_file = guide_file[guide_file['nSample'] <= 8000]

guide_new_file = guide_file[["syllable_path", "syllable_stress_type", "suid"]].groupby('suid').first().reset_index()


path_col = guide_new_file["syllable_path"]
gt_col = guide_new_file["syllable_stress_type"]

In [20]:
ds = SyllableDataset(train_cut_syllable_, 
                            os.path.join(src_eng_, "guide_train_mod.csv"), 
                            select=select, 
                            mapper=mymap, 
                            transform=mytrans)

/home/franklhtan/Projects/proslearn/scripts/model_dataset.py:200: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  guide_file = pd.read_csv(guide_)


In [21]:
use_len = int(0.01 * len(ds))
remain_len = len(ds) - use_len
use_train_ds, remain_ds = random_split(ds, [use_len, remain_len])

In [22]:
len(ds)

12795

In [23]:
use_train_ds.indices

[7344,
 7774,
 7958,
 2649,
 12459,
 9838,
 7959,
 373,
 3833,
 2083,
 9189,
 7941,
 7090,
 3509,
 9305,
 3470,
 1391,
 7102,
 4191,
 8021,
 5057,
 926,
 6971,
 9797,
 3962,
 9205,
 12775,
 5661,
 1537,
 688,
 7822,
 12310,
 3079,
 4568,
 805,
 2602,
 2622,
 1123,
 6488,
 9203,
 10130,
 6046,
 5636,
 6767,
 1106,
 5878,
 11886,
 11809,
 1250,
 768,
 1752,
 12120,
 11552,
 6923,
 9697,
 7044,
 7788,
 9558,
 11008,
 508,
 7780,
 4534,
 3685,
 8968,
 10724,
 6614,
 3981,
 11882,
 1763,
 4102,
 12247,
 2666,
 1058,
 5909,
 6856,
 6339,
 8916,
 7946,
 892,
 4425,
 5615,
 6219,
 6876,
 9247,
 6052,
 2829,
 2863,
 491,
 8388,
 7403,
 11899,
 3680,
 3132,
 1822,
 12388,
 1387,
 7833,
 5798,
 6857,
 7540,
 485,
 3603,
 2326,
 741,
 11781,
 647,
 8259,
 6312,
 5845,
 8682,
 1704,
 10760,
 11465,
 1038,
 8001,
 9727,
 8936,
 8882,
 5143,
 11058,
 11157,
 10193,
 6456,
 7430,
 6463,
 48,
 2478]

In [56]:
train_ds_indices = DS_Tools.read_indices(os.path.join(model_save_, "A1-0605113709-1", f"valid_full.use"))

In [57]:
use_train_ds = torch.utils.data.Subset(ds, train_ds_indices)

In [58]:
use_train_ds.indices

[13581,
 24016,
 20079,
 8498,
 11685,
 31293,
 25064,
 24526,
 20871,
 619,
 12718,
 29509,
 15768,
 15974,
 5001,
 10527,
 31902,
 25379,
 15844,
 32258,
 6800,
 22725,
 18344,
 8371,
 7337,
 31186,
 1225,
 2404,
 19843,
 3589,
 3646,
 25771,
 13027,
 10765,
 13022,
 24,
 27590,
 11008,
 29392,
 7280,
 24660,
 11839,
 11908,
 7795,
 9220,
 24603,
 3153,
 10461,
 8796,
 27175,
 28004,
 10706,
 18528,
 4460,
 19684,
 25667,
 20339,
 20076,
 10985,
 276,
 19354,
 26157,
 7670,
 24567,
 7863,
 10228,
 16503,
 21419,
 22533,
 18900,
 15798,
 27321,
 4287,
 24590,
 32708,
 31288,
 23867,
 26421,
 1005,
 22268,
 2326,
 13539,
 25364,
 20190,
 24383,
 11842,
 30893,
 11293,
 20182,
 32639,
 4149,
 9391,
 10511,
 21034,
 30009,
 30963,
 22026,
 19185,
 18666,
 25658,
 10410,
 18998,
 4783,
 5284,
 10943,
 7193,
 30456,
 25096,
 12896,
 29475,
 18333,
 5430,
 32040,
 12860,
 19913,
 1995,
 6115,
 4621,
 22618,
 16444,
 5665,
 22251,
 12388,
 1978,
 13912,
 20477,
 409,
 17186,
 9273,
 26355,
 

In [ ]:

train_loader = DataLoader(use_train_ds, batch_size=TrainingConfigs.BATCH_SIZE, 
                        shuffle=True, 
                        num_workers=TrainingConfigs.LOADER_WORKER)